In [3]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

import gzip
import numpy as np
import tensorflow as tf
from typing import Tuple

Now that you've learned about tensors, variables, and automatic differentiation, you're ready to learn how to train a neural network using TensorFlow. You've already seen how to train a neural network using Keras in [module 1](../intro-machine-learning-keras) &mdash; in this notebook, we'll re-implement the training loop in TensorFlow. This will help you understand what goes on under the hood a bit better, will give you the opportunity to customize the training loop if you want, and will enable you to debug it.

We'll start by including code that gives us the datasets and model that we'll use in the remainder of this notebook. We will use the same FashionMNIST dataset and data loading code as in [module 1](../intro-machine-learning-keras), so feel free to re-visit that module if something is not clear, or have a look [at the source code](https://github.com/MicrosoftDocs/tensorflow-learning-path/blob/main/intro-keras/kintro.py)

In [26]:
!wget -Nq https://raw.githubusercontent.com/MicrosoftDocs/tensorflow-learning-path/main/intro-keras/kintro.py
from kintro import *

We will be using the following neural network model in our example:

In [27]:
def get_model() -> tf.keras.Model:
  model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(10)
  ])
  return model

As we mentioned in module 1, the goal of training the neural network is to find parameters $W$ and $b$ that minimize the **loss function**, which measures the difference between the actual and predicted labels. We can visualize this in the following diagram:

![Basic neural network with details](images/1-basic-nn-with-details.png)

We also mentioned that we can think of the neural network as the function $\ell$ below, and that we use an optimization algorithm to find the parameters $W$ and $b$ that minimize this function.

$$
\mathrm{loss} = \ell(X, y, W, b)
$$

Let's now dig deeper into what this optimization algorithm might look like. There are many types of optimization algorithms, but in this tutorial we'll cover only the simplest one: the gradient descent algorithm. To implement gradient descent, we iteratively improve our estimates of $W$ and $b$ according to the update formulas below, until the gradients are smaller than a pre-defined threshold $\epsilon$ (or for a pre-defined number of times):

$$
\begin{align}
  W &:= W - \alpha \frac{\partial \ell}{\partial W} \\
  b &:= b - \alpha \frac{\partial \ell}{\partial b}
\end{align}
$$

The parameter $\alpha$ is typically referred to as the "learning rate," and will be defined later in the code. 

When doing training, we pass a mini-batch of data as input, perform a sequence of calculations to obtain the loss, then propagate back through the network to calculate the derivatives used in the gradient descent formulas above. Once we have the derivatives, we can update the values of the network's parameters $W$ and $b$ according to the formulas. This sequence of steps is the backpropagation algorithm. By performing these calculations several times, our parameters get updated repeatedly, getting more and more accurate each time. 

In Keras, when we called the function [`fit`](https://www.tensorflow.org/api_docs/python/tf/keras/Model#fit), the backpropagation algorithm was executed several times. Here, we'll start by understanding the code that reflects a single pass of the backpropagation algorithm:

- a forward pass through the model to compute the predicted value, `y_prime = model(X, training=True)`
- a calculation of the loss using a loss function, `loss = loss_fn(y, y_prime)`
- a backward pass from the loss function through the model to calculate derivatives, `grads = tape.gradient(loss, model.trainable_variables)`
- a gradient descent step to update $W$ and $b$ using the derivatives calculated in the backward pass, `optimizer.apply_gradients(zip(grads, model.trainable_variables))`

Here's the complete code:

In [28]:
def fit_one_batch(X, y, model, loss_fn, optimizer) -> Tuple[tf.Tensor, tf.Tensor]:
  with tf.GradientTape() as tape:
    y_prime = model(X, training=True)
    loss = loss_fn(y, y_prime)

  grads = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(grads, model.trainable_variables))

  return (y_prime, loss)

Notice that the code above ensures that the forward calculations are within the `GradientTape`'s scope, just as we saw in the previous notebook. This makes it possible for us to ask the tape for the gradients. 

The code above works for a single mini-batch, which is typically much smaller than the full set of data (in this sample we use a mini-batch of size 64, out of 60,000 training data items). But we want to execute the backpropagation algorithm for the full set of data. We can do so by iterating through the `Dataset` we created earlier, which, as we saw in module 1, returns a mini-batch per iteration. There are two critical lines in the code below: the `for` loop and the call to the `fit_one_batch` function. The rest of the code just prints the accuracy and loss as the model is being trained. 

In [29]:
def fit(dataset: tf.data.Dataset, model: tf.keras.Model, loss_fn: tf.keras.losses.Loss, 
optimizer: tf.optimizers.Optimizer) -> None:
  batch_count = len(dataset)
  loss_sum = 0
  correct_item_count = 0
  current_item_count = 0
  print_every = 100

  for batch_index, (X, y) in enumerate(dataset):
    (y_prime, loss) = fit_one_batch(X, y, model, loss_fn, optimizer)

    y = tf.cast(y, tf.int64)
    correct_item_count += (tf.math.argmax(y_prime, axis=1) == y).numpy().sum()

    batch_loss = loss.numpy()
    loss_sum += batch_loss
    current_item_count += len(X)

    if ((batch_index + 1) % print_every == 0) or ((batch_index + 1) == batch_count):
      batch_accuracy = correct_item_count / current_item_count * 100
      print(f'[Batch {batch_index + 1:>3d} - {current_item_count:>5d} items] accuracy: {batch_accuracy:>0.1f}%, loss: {batch_loss:>7f}')

A complete iteration over all mini-batches in the dataset is called an "epoch." In this sample, we restrict the code to just two epochs for quick execution, but in a real project you would want to set it to a much higher number (to achieve better predictions). The code below also shows the creation of the loss function and optimizer, which we discussed in module 1.

In [31]:
learning_rate = 0.1
batch_size = 64
epochs = 2

(train_dataset, test_dataset) = get_data(batch_size)

model = get_model()

loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
optimizer = tf.optimizers.SGD(learning_rate)

print('\nFitting:')
for epoch in range(epochs):
  print(f'\nEpoch {epoch + 1}\n-------------------------------')
  fit(train_dataset, model, loss_fn, optimizer)


Fitting:

Epoch 1
-------------------------------
[Batch 100 -  6400 items] accuracy: 66.0%, loss: 0.685200
[Batch 200 - 12800 items] accuracy: 71.5%, loss: 0.637024
[Batch 300 - 19200 items] accuracy: 74.3%, loss: 0.662844
[Batch 400 - 25600 items] accuracy: 76.0%, loss: 0.484288
[Batch 500 - 31968 items] accuracy: 77.2%, loss: 0.515463
[Batch 600 - 38368 items] accuracy: 78.2%, loss: 0.674882
[Batch 700 - 44768 items] accuracy: 78.9%, loss: 0.415960
[Batch 800 - 51168 items] accuracy: 79.5%, loss: 0.366401
[Batch 900 - 57568 items] accuracy: 80.1%, loss: 0.397109
[Batch 938 - 60000 items] accuracy: 80.3%, loss: 0.500174

Epoch 2
-------------------------------
[Batch 100 -  6400 items] accuracy: 84.9%, loss: 0.549428
[Batch 200 - 12800 items] accuracy: 85.1%, loss: 0.281027
[Batch 300 - 19200 items] accuracy: 85.3%, loss: 0.535860
[Batch 400 - 25600 items] accuracy: 85.2%, loss: 0.491699
[Batch 500 - 32000 items] accuracy: 85.3%, loss: 0.400586
[Batch 600 - 38400 items] accuracy: 85

## Demistifying the Model

In this module, we were still using high-level Keras syntax for model definition. However, it is important to understand that the model consists of the same basic concepts that we have already learnt: tensors and variables. For example, a linear `Dense` layer is represented by two variables, `W` and `b`, which contain trainable weights of the layer. The simplest way to define our two-layer dense neural network, equivalent to the one we were using, would be write the following function:  

In [54]:
W1 = tf.Variable(tf.random.normal(shape=(784,512))) # Weight of first layer
b1 = tf.Variable(tf.zeros(shape=(512,))) # Bias of first layer
W2 = tf.Variable(tf.random.normal(shape=(512,512))) # Weight of the second layer
b2 = tf.Variable(tf.zeros(shape=(512))) # Bias of second layer
W3 = tf.Variable(tf.random.normal(shape=(512,10))) # Weight of output layer
b3 = tf.Variable(tf.zeros(shape=(10,))) # Bias of output layer

def mymodel(x):
    x = tf.reshape(x,[-1,784]) # flatten
    x = tf.matmul(x,W1)+b1
    x = tf.nn.relu(x)
    x = tf.matmul(x,W2)+b2
    x = tf.nn.relu(x)
    x = tf.matmul(x,W3)+b3
    return x

random_input = tf.random.normal(shape=(2,28,28))

print(mymodel(random_input))

tf.Tensor(
[[ -6534.038   -2235.3281  -3665.7358   8478.205    4422.8086  -7590.745
   -2849.0298  -1029.7246   2818.633    1850.2557]
 [  6197.5376  -8896.773   -1897.0498  19720.16    -3257.3933 -14803.206
    4768.8804  -7887.8174  -1878.3442   7202.9473]], shape=(2, 10), dtype=float32)


This function is designed to operate on minibatches, in the same way as `model` defined above using Keras syntax. The main difference is that Keras framework encapsulates all weights into corresponsing classes, and makes it easy to compose the final model from a sequence of layers. However, behind the scenes the inner machinery of a neural network is pretty simple.

We can even try to train our simple model using gradient descent:

In [55]:
lr = 0.01 # Learning rate
parameters = [W1,b1,W2,b2,W3,b3]

for i, (X, y) in enumerate(train_dataset):
    # Compute loss and gradients
    with tf.GradientTape() as tape:
        y_prime = mymodel(X)
        loss = tf.keras.losses.sparse_categorical_crossentropy(y, y_prime, from_logits=True)
        loss = tf.reduce_mean(loss) # average the loss over minibatch
        grads = tape.gradient(loss, parameters)
    # Now we need to adjust weights. Instead of calling optimizer.apply_gradients,
    # We will implement our own gradient descent:
    for grad,var in zip(grads,parameters):
        var.assign_sub(lr*grad)
    # Print progress
    if i%100 == 0:
        print(f"Loss = {loss}")

Loss = 4909.4169921875
Loss = 62.308170318603516
Loss = 52.73152160644531
Loss = 26.214557647705078
Loss = 31.049571990966797
Loss = 23.659847259521484
Loss = 25.85222053527832
Loss = 25.639324188232422
Loss = 16.874116897583008
Loss = 24.228927612304688


For simplicity, we do not compute accuracy in this code, but the fact that loss decreases steadily indicates that the network is training.

This example demonstrates that you only need two main things to build a neural network from scratch:
 * efficient operations on tensors, and 
 * the ability to automatically compute gradients. 
 
It is not a problem if you don't immediately understand how this code works - you can always use more high-level concepts like Keras models, existing optimizers, etc. However, it is useful to realize that the main mechanics of a neural network can be programmed in a few lines of low-level code. 